<a href="https://colab.research.google.com/github/qasimriaz002/Pneumonia-Detection-Flask/blob/main/AIH2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import pickle

from flask import Flask, request, render_template
from werkzeug.utils import secure_filename

In [2]:
# Define dataset paths
train_path = "/content/drive/MyDrive/AIH2D/data/train"
test_path = "/content/drive/MyDrive/AIH2D/data/test"

In [3]:
print(train_path)
print(test_path)

/content/drive/MyDrive/AIH2D/data/train
/content/drive/MyDrive/AIH2D/data/test


In [4]:
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Load and preprocess the training data
train_set = train_datagen.flow_from_directory(train_path,
                                              target_size=(64, 64),
                                              batch_size=32,
                                              class_mode='binary')

Found 5233 images belonging to 2 classes.


In [6]:
# Load and preprocess the test data
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

Found 624 images belonging to 2 classes.


In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(train_set, epochs=10, validation_data=test_set)

Epoch 1/10
164/164 [==============================] - 1750s 11s/step - loss: 0.4313 - accuracy: 0.8078 - val_loss: 0.4200 - val_accuracy: 0.8061
Epoch 2/10
164/164 [==============================] - 69s 420ms/step - loss: 0.2647 - accuracy: 0.8959 - val_loss: 0.3094 - val_accuracy: 0.8814
Epoch 3/10
164/164 [==============================] - 68s 415ms/step - loss: 0.2182 - accuracy: 0.9138 - val_loss: 0.3176 - val_accuracy: 0.8702
Epoch 4/10
164/164 [==============================] - 65s 394ms/step - loss: 0.1888 - accuracy: 0.9291 - val_loss: 0.3185 - val_accuracy: 0.8814
Epoch 5/10
164/164 [==============================] - 64s 393ms/step - loss: 0.1652 - accuracy: 0.9367 - val_loss: 0.5100 - val_accuracy: 0.8397
Epoch 6/10
164/164 [==============================] - 65s 396ms/step - loss: 0.1537 - accuracy: 0.9404 - val_loss: 0.3213 - val_accuracy: 0.8670
Epoch 7/10
164/164 [==============================] - 64s 392ms/step - loss: 0.1583 - accuracy: 0.9410 - val_loss: 0.4019 - val_ac

In [9]:
# Evaluate the model
score = model.evaluate(test_set)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

20/20 [==============================] - 5s 265ms/step - loss: 0.3391 - accuracy: 0.8878
Test Loss: 0.33908069133758545
Test Accuracy: 0.8878205418586731


In [10]:
from sklearn.metrics import classification_report

In [12]:
# Evaluate the model
y_true = test_set.classes
y_pred_prob = model.predict(test_set)
y_pred = (y_pred_prob > 0.5).astype(int)

20/20 [==============================] - 7s 316ms/step


In [13]:
# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))


Classification Report:
              precision    recall  f1-score   support

      Normal       0.38      0.30      0.33       234
   Pneumonia       0.63      0.70      0.66       390

    accuracy                           0.55       624
   macro avg       0.50      0.50      0.50       624
weighted avg       0.53      0.55      0.54       624



In [14]:
# Save the model to a file
model.save('pneumonia_model.h5')

# Save the model's training history
with open('history.pickle', 'wb') as file:
    pickle.dump(model.history.history, file)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
import joblib

from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('pneumonia_model.h5')

# Save the model as a pickle file
joblib.dump(model, 'pneumonia_model.pkl')

['pneumonia_model.pkl']

In [18]:

from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [25]:
# Load the trained model
model = load_model('pneumonia_model.h5')

In [26]:
app = Flask(__name__)

In [27]:
# Render the home page
@app.route('/')
def home():
    return render_template('index.html')


In [28]:
# Handle the prediction
@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Get the uploaded image file
        file = request.files['file']
        # Save the uploaded file
        file_path = 'static/' + secure_filename(file.filename)
        file.save(file_path)

        # Preprocess the image
        img = image.load_img(file_path, target_size=(64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img /= 255.0

        # Make predictions
        prediction = model.predict(img)
        result = "Pneumonia" if prediction > 0.5 else "Normal"

        return render_template('index.html', result=result, img_path=file_path)


In [32]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [31]:
from google.colab import output
output.serve_kernel_port_as_window(5000)

<IPython.core.display.Javascript object>